## Welcome to your notebook.


#### Run this cell to connect to your GIS and get started:

In [1]:
from arcgis.gis import GIS, features
gis = GIS("home")

#### Now you are ready to start!

In [5]:
import requests
import json
import numpy as np
import pandas as pd

In [6]:
# otrzymanie listy wszystkich stanowisk pomiarowych
response = requests.get("https://api.gios.gov.pl/pjp-api/rest/station/findAll")
responseJSON = json.loads(response.text)

In [7]:
#print(responseJSON)
#pobranie listy id stacji 
stationId = [station['id'] for i, station in enumerate(responseJSON)] #id stacji
stationLat = [station['gegrLat'] for i, station in enumerate(responseJSON)] # wspolzendne stacji, są one w kolejnosci id stacji więc ok
stationLon = [station['gegrLon'] for i, station in enumerate(responseJSON)]

In [8]:
#pobranie danych dla całej polski
data = []
for i in range(0, len(stationId)):
    link=f"https://api.gios.gov.pl/pjp-api/rest/station/sensors/{stationId[i]}"
    response_s = requests.get(link)
    responseJSON_s = json.loads(response_s.text)
    data.append(responseJSON_s)

In [9]:
#filtracja danych 
#lista stationId, Id (czujnika), paramCode czyli co czujnik miezy 
sensor_list = []

for sublist in data:
    for item in sublist:
        # Wyodrębnij potrzebne informacje
        station_id = item['stationId']
        sensor_id = item['id']
        param_code = item['param']['paramCode']

        # Dodaj do listy wynikowej
        sensor_list.append([station_id, sensor_id, param_code])

# Wyświetl uzyskaną listę
for row in sensor_list:
    print(row)

[114, 642, 'NO2']
[114, 644, 'O3']
[117, 672, 'SO2']
[117, 658, 'C6H6']
[117, 660, 'CO']
[117, 665, 'NO2']
[117, 667, 'O3']
[117, 670, 'PM2.5']
[117, 14395, 'PM10']
[129, 737, 'CO']
[129, 740, 'NO2']
[129, 744, 'PM2.5']
[52, 282, 'C6H6']
[52, 285, 'CO']
[52, 291, 'NO2']
[52, 293, 'O3']
[52, 297, 'SO2']
[52, 14397, 'PM10']
[109, 605, 'C6H6']
[109, 608, 'CO']
[109, 614, 'NO2']
[109, 616, 'O3']
[109, 618, 'PM10']
[109, 621, 'SO2']
[11, 50, 'NO2']
[11, 52, 'O3']
[11, 54, 'PM10']
[16, 101, 'PM10']
[38, 221, 'NO2']
[38, 223, 'O3']
[38, 224, 'PM10']
[38, 25988, 'PM2.5']
[70, 397, 'NO2']
[70, 400, 'PM10']
[70, 20214, 'O3']
[74, 441, 'NO2']
[74, 444, 'O3']
[74, 450, 'SO2']
[9153, 14727, 'NO2']
[9153, 14729, 'O3']
[9153, 14730, 'PM10']
[9153, 14731, 'PM2.5']
[9153, 14733, 'SO2']
[9153, 14734, 'C6H6']
[9153, 14707, 'CO']
[11254, 20210, 'PM10']
[11934, 21368, 'NO2']
[11934, 21370, 'PM10']
[11934, 21371, 'O3']
[12055, 29112, 'NO2']
[12055, 21628, 'O3']
[12055, 21636, 'PM10']
[156, 949, 'C6H6']
[156

In [10]:
#sortowanie wynikow
sensor_NO2 = []
sensor_PM10 = []
sensor_PM25 = []

sensor_NO2 = [row for row in sensor_list if row[2] == 'NO2']
sensor_PM10 = [row for row in sensor_list if row[2] == 'PM10']
sensor_PM25 = [row for row in sensor_list if row[2] == 'PM2.5']

print(sensor_NO2)

[[114, 642, 'NO2'], [117, 665, 'NO2'], [129, 740, 'NO2'], [52, 291, 'NO2'], [109, 614, 'NO2'], [11, 50, 'NO2'], [38, 221, 'NO2'], [70, 397, 'NO2'], [74, 441, 'NO2'], [9153, 14727, 'NO2'], [11934, 21368, 'NO2'], [12055, 29112, 'NO2'], [156, 959, 'NO2'], [158, 991, 'NO2'], [295, 2064, 'NO2'], [296, 2083, 'NO2'], [314, 2195, 'NO2'], [355, 2373, 'NO2'], [10058, 16231, 'NO2'], [291, 2031, 'NO2'], [319, 2219, 'NO2'], [322, 2242, 'NO2'], [331, 16638, 'NO2'], [9913, 16037, 'NO2'], [590, 16160, 'NO2'], [568, 3839, 'NO2'], [584, 21351, 'NO2'], [16913, 27813, 'NO2'], [671, 4383, 'NO2'], [10125, 16319, 'NO2'], [638, 4192, 'NO2'], [659, 4278, 'NO2'], [665, 4333, 'NO2'], [882, 16424, 'NO2'], [10005, 16216, 'NO2'], [10030, 16181, 'NO2'], [944, 6081, 'NO2'], [16493, 26990, 'NO2'], [952, 6161, 'NO2'], [902, 5899, 'NO2'], [920, 5979, 'NO2'], [946, 6104, 'NO2'], [950, 6140, 'NO2'], [10834, 17957, 'NO2'], [986, 6348, 'NO2'], [987, 6379, 'NO2'], [805, 5281, 'NO2'], [813, 5346, 'NO2'], [814, 5371, 'NO2'], [

In [11]:
#pobranie danych z czujnikow


data_NO2 = [] #wartosci z czujnikow
data_PM10 = []
data_PM25 = []

for i in range(0, len(sensor_NO2)):
    link=f"https://api.gios.gov.pl/pjp-api/rest/data/getData/{sensor_NO2[i][1]}"
    response_n = requests.get(link)
    responseJSON_n = json.loads(response_n.text)
    data_NO2.append([sensor_NO2[i][0], responseJSON_n])#dodaje tu jeszcze do wynikow id stacji, zeby latwo powiazac ze wspolrzednymi 
    
for i in range(0, len(sensor_PM10)):
    link=f"https://api.gios.gov.pl/pjp-api/rest/data/getData/{sensor_PM10[i][1]}"
    response_10 = requests.get(link)
    responseJSON_10 = json.loads(response_10.text)
    data_PM10.append([sensor_PM10[i][0], responseJSON_10])
    
for i in range(0, len(sensor_PM25)):
    link=f"https://api.gios.gov.pl/pjp-api/rest/data/getData/{sensor_PM25[i][1]}"
    response_25 = requests.get(link)
    responseJSON_25 = json.loads(response_25.text)
    data_PM25.append([sensor_PM25[i][0], responseJSON_25])

In [60]:
miasta = []  #miasta

for i in range(len(stationId)):
    miasta.append(responseJSON[i]['city']['name'])

In [61]:
#usuwanie braku pomiarów
for i in range(len(data_PM10) - 1, -1, -1):
    if data_PM10[i][1]['values'] == []:
        data_PM10.pop(i)

In [62]:
for i in range(len(data_NO2) - 1, -1, -1):
    if data_NO2[i][1]['values'] == []:
        data_NO2.pop(i)

In [63]:
for i in range(len(data_PM25) - 1, -1, -1):
    if data_PM25[i][1]['values'] == []:
        data_PM25.pop(i)

In [64]:
data_PM10_df = pd.DataFrame({'ID_stacji' : [d[0] for d in data_PM10], 'Dat_p_pm10' : [data_PM10[i][1]['values'][1]['date'] for i in range(0, len(data_PM10))], 'Pom_pm10' : [data_PM10[i][1]['values'][1]['value'] for i in range(0, len(data_PM10))]}) 

In [65]:
data_NO2_df = pd.DataFrame({'ID_stacji' : [d[0] for d in data_NO2], 'Dat_p_no2' : [data_NO2[i][1]['values'][1]['date'] for i in range(0, len(data_NO2))], 'Pom_no2' : [data_NO2[i][1]['values'][1]['value'] for i in range(0, len(data_NO2))]}) 

In [66]:
data_PM25_df = pd.DataFrame({'ID_stacji' : [d[0] for d in data_PM25], 'Dat_p_pm25' : [data_PM25[i][1]['values'][1]['date'] for i in range(0, len(data_PM25))], 'Pom_pm25' : [data_PM25[i][1]['values'][1]['value'] for i in range(0, len(data_PM25))]}) 

In [67]:
miasta_df = pd.DataFrame({'ID_stacji' : stationId, 'Miasto' : miasta})

In [68]:
stations = pd.DataFrame({'ID_stacji' : stationId, 'Lat_stacji' : stationLat, 'Lon_stacji' : stationLon})

In [69]:
stations = stations.join(data_PM10_df.set_index('ID_stacji'), on='ID_stacji', lsuffix='id')

In [70]:
stations = stations.join(data_NO2_df.set_index('ID_stacji'), on='ID_stacji', lsuffix='id')

In [71]:
stations = stations.join(data_PM25_df.set_index('ID_stacji'), on='ID_stacji', lsuffix='id')

In [72]:
stations = stations.join(miasta_df.set_index('ID_stacji'), on='ID_stacji', lsuffix='id')

In [73]:
convert_dict = {'ID_stacji': int, 'Lat_stacji': float, 'Lon_stacji': float, 'Dat_p_pm25' : str, 'Pom_pm25' : float, 'Dat_p_no2' : str, 'Pom_no2' : float, 'Dat_p_pm10' : str, 'Pom_pm10' : float}
 
stations = stations.astype(convert_dict)

In [74]:
stations

,ID_stacji,Lat_stacji,Lon_stacji,Dat_p_pm10,Pom_pm10,Dat_p_no2,Pom_no2,Dat_p_pm25,Pom_pm25,Miasto
0,114,51.115933,17.141125,nan,NaN,2024-01-04 15:00:00,9.16326,nan,NaN,Wrocław
1,117,51.129378,17.029250,2024-01-04 15:00:00,3.53310,2024-01-04 15:00:00,9.07456,2024-01-04 15:00:00,2.87119,Wrocław
2,129,51.086225,17.012689,nan,NaN,2024-01-04 15:00:00,40.91360,2024-01-04 15:00:00,2.94037,Wrocław
3,52,51.204503,16.180513,2024-01-04 15:00:00,4.16353,2024-01-04 15:00:00,9.71880,nan,NaN,Legnica
4,109,50.768729,16.269677,2024-01-04 15:00:00,2.51577,2024-01-04 15:00:00,2.86056,nan,NaN,Wałbrzych
...,...,...,...,...,...,...,...,...,...,...
198,11336,54.194114,16.211672,nan,NaN,2024-01-04 15:00:00,10.71950,nan,NaN,Koszalin
199,961,53.122325,14.382245,nan,NaN,2024-01-04 15:00:00,4.26482,nan,NaN,Widuchowa
200,983,53.698902,16.704556,2024-01-04 15:00:00,5.59678,2024-01-04 15:00:00,14.76680,nan,NaN,Szczecinek
201,10934,54.179324,15.596342,2024-01-04 15:00:00,3.51893,nan,NaN,2024-01-04 15:00:00,2.44680,Kołobrzeg


In [75]:
sedf = pd.DataFrame.spatial.from_xy(stations, 'Lon_stacji', 'Lat_stacji')

In [76]:
lyr1 = sedf.spatial.to_featurelayer('pomiar1')

In [ ]:
lyr2 = sedf.spatial.to_featurelayer('pomiar2')

In [ ]:
lyr3 = sedf.spatial.to_featurelayer('pomiar3')

In [ ]:
lyr1

In [ ]:
lyr2

In [ ]:
lyr3